# Automatically downloading real estate data

The Mortgage Bankers Association sends a press release every week with the results of their weekly mortgage applications survey. The survey covers over 75% of all applications for retail residential mortgages in the U.S., and contains a huge wealth of information, including interest rates and changes in the number of applications. These press releases are all available on [one page](https://www.mba.org/news-research-and-resources/research-and-economics/single-family-research/weekly-applications-survey/research-and-economics-all-news-about-mbas-weekly-applications-survey) and go back to 2016. The survey itself has been conducted since 1990. Find more information about the survey [here](https://www.mba.org/Documents/mba.org/files/Research/HistoricalWAS/WASFAQ.pdf). 

The purpose of this script is to automatically download all of the press releases and extract information about interest rates.

## Part 1: Get a list of URLs

I need to find the URLs of each press release on the webpage and put them into a list.

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.mba.org/news-research-and-resources/research-and-economics/single-family-research/weekly-applications-survey/research-and-economics-all-news-about-mbas-weekly-applications-survey'

headers = {'name': "Sharon Lurye", 
           'email': "sharonrlurye@gmail.com",
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

response = requests.get(url, headers=headers)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

We want to find this part:

```
<ul class="item-list search-results-list">
<h1 class="schema-pressrelease prop-resourcename item-title search-results-item-title"> <a href="http://www.mba.org/2021-press-releases/june/mortgage-applications-increase-in-latest-mba-weekly-survey-x281353">                 Mortgage Applications Increase in Latest MBA Weekly Survey                 </a> </h1>
<p> <span class="schema-pressrelease prop-articledate">                 June 23, 2021                 </span></p>
<br/>
```

In [4]:
links = soup.select(".search-results-list a")

links = [link['href'] for link in links]

In [5]:
len(links)

248

## Part 2: Find the date of each survey and the interest rates

Let's start with the first press release only.

In [14]:
links[0]

'http://www.mba.org/2021-press-releases/june/mortgage-applications-decrease-in-latest-mba-weekly-survey-x281049'

In [16]:
url = links[0]

headers = {'name': "Sharon Lurye", 'email': "sharonrlurye@gmail.com"}

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, "html.parser")


In [18]:
#Find the title of the press release
soup.find('h1', attrs={'class':'title-primary'}).text.strip()

'Mortgage Applications Decrease in Latest MBA Weekly Survey'

In [19]:
#Find first two paragraphs of the press release
soup.find_all('p')[0:3]

[<p></p>,
 <p><strong></strong><strong>WASHINGTON, D.C. (June 30, 2021)</strong> - <strong>Mortgage applications decreased 6.9 percent from one week earlier, </strong>according to data from the Mortgage Bankers Association's (MBA) Weekly Mortgage Applications Survey for the week ending June 25, 2021. </p>,
 <p>The Market Composite Index, a measure of mortgage loan application volume, decreased 6.9 percent on a seasonally adjusted basis from one week earlier. On an unadjusted basis, the Index decreased 7 percent compared with the previous week. The Refinance Index decreased 8 percent from the previous week and was 15 percent lower than the same week one year ago. The seasonally adjusted Purchase Index decreased 5 percent from one week earlier. The unadjusted Purchase Index decreased 6 percent compared with the previous week and was 17 percent lower than the same week one year ago.  </p>]

In [21]:
#Find text of the first paragraph
soup.find_all('p')[1].text.strip()

"WASHINGTON, D.C. (June 30, 2021) -\xa0Mortgage applications decreased 6.9 percent from one week earlier, according to data from the Mortgage Bankers Association's (MBA) Weekly Mortgage Applications Survey for the week ending June 25, 2021."

In [28]:
#https://www.programiz.com/python-programming/regex#python-regex
#https://stackoverflow.com/questions/35413746/regex-to-match-date-like-month-name-day-comma-and-year/35413952
#https://stackoverflow.com/questions/59219456/extract-month-names-and-date-numbers-from-a-raw-string-using-regex-edit-new-te

import re

string = soup.find_all('p')[1].text
pattern = "ending (\w+ \d+, \d\d\d\d)."

#I want it to return June 18 (week the surveyended) but NOT June 23 (week the press release came out) so I search for the date with a whitespace in front. Then I use strip() to remove that whitespace.

result = re.findall(pattern, string)
print(result[0].strip())

June 25, 2021


In [29]:
#Print all paragraphs that mention interest rates
#https://stackabuse.com/python-check-if-string-contains-substring 

string = "interest rate"

#Grab date from the first paragraph
for paragraph in soup.find_all('p'):
    if string in paragraph.text:
        print(paragraph.text, "\n")

The average contract interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) increased to 3.20 percent from 3.18 percent, with points decreasing to 0.39 from 0.48 (including the origination fee) for 80 percent loan-to-value ratio (LTV) loans. The effective rate remained unchanged from last week. 

The average contract interest rate for 30-year fixed-rate mortgages with jumbo loan balances (greater than $548,250) decreased to 3.23 percent from 3.26 percent, with points decreasing to 0.33 from 0.44 (including the origination fee) for 80 percent LTV loans. The effective rate decreased from last week.   The average contract interest rate for 30-year fixed-rate mortgages backed by the FHA decreased to 3.19 percent from 3.21 percent, with points remaining unchanged at 0.34 (including the origination fee) for 80 percent LTV loans. The effective rate decreased from last week.   The average contract interest rate for 15-year fixed-rate mortgages decreased

In [30]:
#Print contract interest rate for 30-year fixed-rate mortgages under $548,250

string = "The average contract interest rate for 30-year fixed-rate mortgages with conforming loan balances"
pattern = "(\d.\d\d) percent"

#Grab date from the first paragraph
for paragraph in soup.find_all('p'):
    if string in paragraph.text:
        print(paragraph.text)
        result = re.findall(pattern, paragraph.text)
        print(result[0])

The average contract interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) increased to 3.20 percent from 3.18 percent, with points decreasing to 0.39 from 0.48 (including the origination fee) for 80 percent loan-to-value ratio (LTV) loans. The effective rate remained unchanged from last week.
3.20


In [31]:
#Print contract interest rate for 5/1 ARMs

string = "The average contract interest rate for 5/1 ARMs"
pattern = "(\d.\d\d) percent"

#Grab date from the first paragraph
for paragraph in soup.find_all('p'):
    if string in paragraph.text:
        print(paragraph.text)
        result = re.findall(pattern, paragraph.text)
        print(result[0])

The average contract interest rate for 30-year fixed-rate mortgages with jumbo loan balances (greater than $548,250) decreased to 3.23 percent from 3.26 percent, with points decreasing to 0.33 from 0.44 (including the origination fee) for 80 percent LTV loans. The effective rate decreased from last week.   The average contract interest rate for 30-year fixed-rate mortgages backed by the FHA decreased to 3.19 percent from 3.21 percent, with points remaining unchanged at 0.34 (including the origination fee) for 80 percent LTV loans. The effective rate decreased from last week.   The average contract interest rate for 15-year fixed-rate mortgages decreased to 2.56 percent from 2.58 percent, with points decreasing to 0.37 from 0.39 (including the origination fee) for 80 percent LTV loans. The effective rate decreased from last week.   The average contract interest rate for 5/1 ARMs increased to 2.98 percent from 2.69 percent, with points decreasing to 0.23 from 0.26 (including the originat

## Create a dataframe with the first 9 links

In [33]:
import pandas as pd

df = pd.DataFrame(links[:9], columns=['url'])
df

,url
0,http://www.mba.org/2021-press-releases/june/mo...
1,http://www.mba.org/2021-press-releases/june/mo...
2,http://www.mba.org/2021-press-releases/june/mo...
3,http://www.mba.org/2021-press-releases/june/mo...
4,http://www.mba.org/2021-press-releases/june/mo...
5,http://www.mba.org/2021-press-releases/may/mor...
6,http://www.mba.org/2021-press-releases/may/mor...
7,http://www.mba.org/2021-press-releases/may/mor...
8,http://www.mba.org/2021-press-releases/may/mor...


In [34]:
df['page_contents'] = df.url.apply(lambda url: requests.get(url).text)
df

,url,page_contents
0,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""..."
1,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""..."
2,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""..."
3,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""..."
4,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""..."
5,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""..."
6,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""..."
7,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""..."
8,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""..."


In [36]:
df['title'] = df.page_contents.apply(lambda contents: BeautifulSoup(contents, 'html.parser').select_one("h1.title-primary").text.strip())
df

,url,page_contents,title
0,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...
1,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...
2,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...
3,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...
4,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...
5,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...
6,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...
7,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...
8,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...


In [37]:
df['survey_date'] = df.page_contents.str.extract("ending (\w+ \d+, \d\d\d\d).")
df

,url,page_contents,title,survey_date
0,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"June 25, 2021"
1,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"June 18, 2021"
2,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"June 11, 2021"
3,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"June 4, 2021"
4,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"May 28, 2021"
5,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"May 21, 2021"
6,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"May 14, 2021"
7,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"May 7, 2021"
8,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"April 30, 2021"


In [38]:
pattern = "interest rate for 30-year fixed-rate mortgages with conforming .* (\d.\d\d) percent"

df['fixed_rates'] = df.page_contents.str.extract(pattern)

df

,url,page_contents,title,survey_date,fixed_rates
0,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"June 25, 2021",3.18
1,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"June 18, 2021",3.11
2,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"June 11, 2021",3.15
3,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"June 4, 2021",3.17
4,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"May 28, 2021",2.53
5,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"May 21, 2021",3.15
6,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"May 14, 2021",3.11
7,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"May 7, 2021",3.18
8,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"April 30, 2021",3.17


In [39]:
pattern = "interest rate for 5/1 ARMs .* (\d.\d\d) percent"

df['arms_rates'] = df.page_contents.str.extract(pattern)

df

,url,page_contents,title,survey_date,fixed_rates,arms_rates
0,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"June 25, 2021",3.18,2.69
1,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"June 18, 2021",3.11,2.69
2,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"June 11, 2021",3.15,2.54
3,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"June 4, 2021",3.17,2.54
4,http://www.mba.org/2021-press-releases/june/mo...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"May 28, 2021",2.53,2.81
5,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"May 21, 2021",3.15,2.58
6,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"May 14, 2021",3.11,2.57
7,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Increase in Latest MBA W...,"May 7, 2021",3.18,2.76
8,http://www.mba.org/2021-press-releases/may/mor...,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""...",Mortgage Applications Decrease in Latest MBA W...,"April 30, 2021",3.17,2.59


# Put it all together

Now I will pull all of the links. This is the complete code.

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [12]:
pd.set_option("display.max_colwidth", 250)

In [55]:
df = pd.DataFrame(links, columns=['url'])
df['page_contents'] = df.url.apply(lambda url: requests.get(url).text)
df['title'] = df.page_contents.apply(lambda contents: BeautifulSoup(contents, 'html.parser').select_one("h1.title-primary").text.strip())
df['survey_date'] = df.page_contents.str.extract("ending (\w+ \d+, \d\d\d\d).")

pattern = "interest rate for 30-year fixed-rate mortgages with conforming .*?(\d.\d\d) percent from"
df['fixed_rates'] = df.page_contents.str.extract(pattern)

pattern = "interest rate for 5/1 ARMs .*? to (\d.\d\d) percent from"
df['arms_rates'] = df.page_contents.str.extract(pattern)

pattern = "(interest rate for 30-year fixed-rate mortgages with conforming .*?\d.\d\d percent from)"
df['fixed_rates_full'] = df.page_contents.str.extract(pattern)

pattern = "(interest rate for 5/1 ARMs .*? \d.\d\d percent from)"
df['arms_rates_full'] = df.page_contents.str.extract(pattern)

df

,url,page_contents,title,survey_date,fixed_rates,arms_rates,fixed_rates_full,arms_rates_full
0,http://www.mba.org/2021-press-releases/june/mortgage-applications-decrease-in-latest-mba-weekly-survey-x281049,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""no-js"">\r\n<head>\r\n <!-- Google Tag Manager -->\r\n <script>(function (w, d, s, l, i) { w[l] = w[l] || []; w[l].push({ 'gtm.start': new Date().getTime(), event: 'gtm.js' }); var f = d.getElementsB...",Mortgage Applications Decrease in Latest MBA Weekly Survey,"June 25, 2021",3.20,2.98,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) increased to 3.20 percent from",interest rate for 5/1 ARMs increased to 2.98 percent from
1,http://www.mba.org/2021-press-releases/june/mortgage-applications-increase-in-latest-mba-weekly-survey-x281353,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""no-js"">\r\n<head>\r\n <!-- Google Tag Manager -->\r\n <script>(function (w, d, s, l, i) { w[l] = w[l] || []; w[l].push({ 'gtm.start': new Date().getTime(), event: 'gtm.js' }); var f = d.getElementsB...",Mortgage Applications Increase in Latest MBA Weekly Survey,"June 18, 2021",3.18,NaN,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) increased to 3.18 percent from",NaN
2,http://www.mba.org/2021-press-releases/june/mortgage-applications-increase-in-latest-mba-weekly-survey,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""no-js"">\r\n<head>\r\n <!-- Google Tag Manager -->\r\n <script>(function (w, d, s, l, i) { w[l] = w[l] || []; w[l].push({ 'gtm.start': new Date().getTime(), event: 'gtm.js' }); var f = d.getElementsB...",Mortgage Applications Increase in Latest MBA Weekly Survey,"June 11, 2021",3.11,2.69,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) decreased to 3.11 percent from",interest rate for 5/1 ARMs increased to 2.69 percent from
3,http://www.mba.org/2021-press-releases/june/mortgage-applications-decrease-in-latest-mba-weekly-survey-x280704,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""no-js"">\r\n<head>\r\n <!-- Google Tag Manager -->\r\n <script>(function (w, d, s, l, i) { w[l] = w[l] || []; w[l].push({ 'gtm.start': new Date().getTime(), event: 'gtm.js' }); var f = d.getElementsB...",Mortgage Applications Decrease in Latest MBA Weekly Survey,"June 4, 2021",3.15,NaN,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) decreased to 3.15 percent from",NaN
4,http://www.mba.org/2021-press-releases/june/mortgage-applications-decrease-in-latest-mba-weekly-survey,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""no-js"">\r\n<head>\r\n <!-- Google Tag Manager -->\r\n <script>(function (w, d, s, l, i) { w[l] = w[l] || []; w[l].push({ 'gtm.start': new Date().getTime(), event: 'gtm.js' }); var f = d.getElementsB...",Mortgage Applications Decrease in Latest MBA Weekly Survey,"May 28, 2021",3.17,2.54,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) decreased to 3.17 percent from",interest rate for 5/1 ARMs decreased to 2.54 percent from
...,...,...,...,...,...,...,...,...
243,http://www.mba.org/2016-press-releases/january/mortgage-applications-increase-as-rates-continue-to-drop-in-latest-mba-weekly-survey,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""no-js"">\r\n<head>\r\n <!-- Google Tag Manager -->\r\n <script>(function (w, d, s, l, i) { w[l] = w[l] || []; w[l].push({ 'gtm.start': new Date().getTime(), event: 'gtm.js' }); var f = d.getElementsB...",Mortgage Applications Increase as Rates Continue to Drop in Latest MBA Weekly Survey,"January 22, 2016",NaN,3.09,NaN,interest rate for 5/1 ARMs decreased to 3.09 percent from
244,http://www.mba.org/2016-press-releases/january/refinance-mortgage-applications-increase-as-rates-fall-in-latest-mba-weekly-survey,"<!DOCTYPE html>\r\n\r\n<html lang=""en"" class=""no-js"">\r\n<head>\r\n <!-- Google Tag Manager -->\r\n 

In [36]:
df2 = df
del df2['page_contents']
df2

,url,title,survey_date,fixed_rates,arms_rates,fixed_rates_full,arms_rates_full
0,http://www.mba.org/2021-press-releases/june/mortgage-applications-decrease-in-latest-mba-weekly-survey-x281049,Mortgage Applications Decrease in Latest MBA Weekly Survey,"June 25, 2021",3.20,2.98,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) increased to 3.20 percent from",interest rate for 5/1 ARMs increased to 2.98 percent from
1,http://www.mba.org/2021-press-releases/june/mortgage-applications-increase-in-latest-mba-weekly-survey-x281353,Mortgage Applications Increase in Latest MBA Weekly Survey,"June 18, 2021",3.18,NaN,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) increased to 3.18 percent from",NaN
2,http://www.mba.org/2021-press-releases/june/mortgage-applications-increase-in-latest-mba-weekly-survey,Mortgage Applications Increase in Latest MBA Weekly Survey,"June 11, 2021",3.11,2.69,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) decreased to 3.11 percent from",interest rate for 5/1 ARMs increased to 2.69 percent from
3,http://www.mba.org/2021-press-releases/june/mortgage-applications-decrease-in-latest-mba-weekly-survey-x280704,Mortgage Applications Decrease in Latest MBA Weekly Survey,"June 4, 2021",3.15,NaN,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) decreased to 3.15 percent from",NaN
4,http://www.mba.org/2021-press-releases/june/mortgage-applications-decrease-in-latest-mba-weekly-survey,Mortgage Applications Decrease in Latest MBA Weekly Survey,"May 28, 2021",3.17,2.54,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($548,250 or less) decreased to 3.17 percent from",interest rate for 5/1 ARMs decreased to 2.54 percent from
...,...,...,...,...,...,...,...
243,http://www.mba.org/2016-press-releases/january/mortgage-applications-increase-as-rates-continue-to-drop-in-latest-mba-weekly-survey,Mortgage Applications Increase as Rates Continue to Drop in Latest MBA Weekly Survey,"January 22, 2016",NaN,3.09,NaN,interest rate for 5/1 ARMs decreased to 3.09 percent from
244,http://www.mba.org/2016-press-releases/january/refinance-mortgage-applications-increase-as-rates-fall-in-latest-mba-weekly-survey,Refinance Mortgage Applications Increase as Rates Fall in Latest MBA Weekly Survey,"January 15, 2016",NaN,3.20,NaN,interest rate for 5/1 ARMs increased to 3.20 percent from
245,http://www.mba.org/2016-press-releases/january/mortgage-applications-increase-in-latest-mba-weekly-survey,Mortgage Applications Increase in Latest MBA Weekly Survey,"January 8, 2016",4.12,3.14,"interest rate for 30-year fixed-rate mortgages with conforming loan balances ($417,000 or less) decreased to 4.12 percent from",interest rate for 5/1 ARMs decreased to 3.14 percent from
246,http://www.mba.org/2016-press-releases/january/mortgage-credit-availability-decreased-in-december,Mortgage Credit Availability Decreased in December,NaN,NaN,NaN,NaN,NaN


In [57]:
df2.to_csv("mortgage_data.csv")